In [1]:
import numpy as np
import random


# Objective: Move from 'S' to 'G', without being in trap and hitting the walls

Define objective: reach goal, avoid traps.

Maze is environment.

Agent interacts with environment.

State = current position.

Actions = up, down, left, right.

Rewards given based on result.

Agent tries actions.

Gets reward.

Q-table updated.

Policy chooses best action using Q-table.

Agent balances exploration and exploitation.

Eventually learns optimal path.


# Reinforcement Learning: Maze Example

---

## 1. Problem Scenario (Motivation)

We consider a maze where an agent must move from a **Start** position to a **Goal**, while avoiding traps and minimizing unnecessary movement.

### Example Maze


### Legend

- `S` = Start
- `G` = Goal
- `X` = Trap
- `.` = Normal cell

### Goal of the Agent

- Reach the goal
- Avoid traps
- Take the shortest safe path

---

## 2. RL Concepts in Maze

| RL Term | Maze Meaning |
|----------|-------------|
| Agent | Player/robot/dog/program |
| Environment | Maze grid |
| State | Current position `(row, col)` |
| Action | Up, Down, Left, Right |
| Reward | Feedback after move |
| Episode | One complete run |
| Policy | Rule for choosing action |
| Q-value | Quality of action in a state |

---

## 3. Reward Structure

### Example Reward Design

| Situation | Reward |
|------------|--------|
| Reach Goal | +10 |
| Enter Trap | −10 |
| Normal move | −1 |

### Why −1 per move?

- Encourages shorter paths
- Punishes unnecessary wandering

---

## 4. What is a Q-value?

A **Q-value** answers:

> "If I take action A in state S, how good will my future rewards be?"

### Example
Q((1,2), right) = 5 

Meaning:

If the agent is at `(1,2)` and moves right, the expected future outcome is good.

---

## 5. What is Q-function?

The **Q-function** calculates and updates action values:

Q(s, a)

Where:
- `s` = state
- `a` = action

It updates after every step:
New Q = Old Q + learning update

Conceptually:
New knowledge = Old knowledge + new experience

---

## 6. What is Q-table?

The **Q-table** stores Q-values for all state–action pairs.

### Example

| State | Up | Down | Left | Right |
|-------|----|------|------|-------|
| (0,0) | 0 | -1 | 0 | 2 |
| (0,1) | 1 | -3 | 0 | 4 |

The agent chooses the action with the **highest value**.

---

## 7. Learning Process Overview

- Agent starts with Q-table full of zeros.
- Moves randomly.
- Receives rewards.
- Q-table updates after every step.
- Good actions get higher values.
- Bad actions get lower values.
- Agent gradually learns optimal path.

---

## 8. Full Episode Example

Consider movement:
A → right → B → down → C → right → Trap

### Rewards Received


-1, -1, -10


### Step-by-Step Learning

#### Step 1: Trap Reached

At state C:

C → right → Trap
Reward = -10

Update:

Q(C, right) becomes strongly negative

Agent learns:

Right from C is bad.

#### Step 2: Previous Step

At B:

B → down → C

Since C leads to trap:

Q(B, down) becomes negative

Agent learns:
Down from B is dangerous.
#### Step 3: Earlier Step

At A:

A → right → B

Since B leads to bad future:

Q(A, right) decreases slightly


Punishment spreads backward through previous states.

---

## 9. Next Episode Behavior

Now from state B:

| Action | Q value |
|--------|---------|
| down | negative |
| others | near zero |

Agent avoids moving down.

---

## 10. Exploration vs Exploitation

Agent must balance:

### Exploration
Try random moves to discover new paths.

### Exploitation
Use the best known action.

### Common Strategy

80% best action
20% random action


Learning continues while still exploring.

---

## 11. How Learning Spreads

Learning spreads backward from outcomes:<br>


Goal discovered <br>
↓<br>
Neighbor states improve<br>
↓<br>
Farther states improve<br>
↓<br>
Full path learned<br>


---

## 12. Final Outcome

After many episodes:

- Bad paths become negative
- Good paths become positive
- Agent chooses shortest safe path

### Example Learned Path




In [16]:
maze = [
    ['S', '.', '.', '.'],
    ['.', 'X', '.', '.'],
    ['.', '.', '.', 'G']
]

ROWS = len(maze)
COLS = len(maze[0])

start_state = (0, 0)
goal_state = (2, 3)
trap_state = (1, 1)


In [41]:
actions = ['up', 'down', 'left', 'right']
action_map = {
    'up': (-1, 0),
    'down': (1, 0),
    'left': (0, -1),
    'right': (0, 1)
}


In [36]:
Q = {}

for r in range(ROWS):
    for c in range(COLS):
        Q[(r, c)] = {a: 0 for a in actions}
Q

{(0, 0): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (0, 1): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (0, 2): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (0, 3): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (1, 0): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (1, 1): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (1, 2): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (1, 3): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (2, 0): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (2, 1): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (2, 2): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (2, 3): {'up': 0, 'down': 0, 'left': 0, 'right': 0}}

In [31]:
alpha = 0.1      # learning rate
gamma = 0.8      # discount factor
epsilon = 0.2    # exploration rate
episodes = 500


In [32]:
def get_reward(state):
    if state == goal_state:
        return 10
    if state == trap_state:
        return -10
    return -1


In [33]:
def get_next_state(state, action):
    r, c = state
    dr, dc = action_map[action]
    nr, nc = r + dr, c + dc

    if nr < 0 or nr >= ROWS or nc < 0 or nc >= COLS:
        return state  # hit wall

    return (nr, nc)


In [40]:
for episode in range(episodes):
    
    state = start_state  

    while state != goal_state and state != trap_state:
        # ε-greedy policy (eXPLORE OR eXPLOIT)
        if random.uniform(0, 1) < epsilon:
            action = random.choice(actions)
        else:
            action = max(Q[state], key=Q[state].get)

        next_state = get_next_state(state, action)
        reward = get_reward(next_state)

        best_next_action = max(Q[next_state], key=Q[next_state].get)

        # Q-learning update
        Q[state][action] += alpha * (
            reward + gamma * Q[next_state][best_next_action] - Q[state][action]
        )

        state = next_state


{(0, 0): {'up': -0.08664124313327089, 'down': 1.1439999999999904, 'left': -0.09192202944538838, 'right': 0.731768982768165}, (0, 1): {'up': -1.0846866885068414, 'down': -4.68559, 'left': -0.7814123476101481, 'right': 2.50032128847682}, (0, 2): {'up': 0.22380109958432445, 'down': 4.558383814210251, 'left': -0.3981732289137754, 'right': -0.2971646711341819}, (0, 3): {'up': -0.198, 'down': 1.9246366259044003, 'left': -0.02844673899017347, 'right': -0.19802745007999997}, (1, 0): {'up': -0.08980977617990463, 'down': 2.679999999999989, 'left': 1.1358123075350133, 'right': -9.962428978738636}, (1, 1): {'up': 0, 'down': 0, 'left': 0, 'right': 0}, (1, 2): {'up': 0.799084519782212, 'down': 6.999538879443562, 'left': -5.217031, 'right': 2.038280305640265}, (1, 3): {'up': -0.1, 'down': 7.712320754503899, 'left': 1.2269689654716784, 'right': 0.4210572479200001}, (2, 0): {'up': 1.1341709349077327, 'down': 2.6550353349719398, 'left': 2.6704641049173903, 'right': 4.599999999999989}, (2, 1): {'up': -9.

In [35]:
print("Learned Policy:")
for r in range(ROWS):
    for c in range(COLS):
        if (r, c) == goal_state:
            print(" G ", end="")
        elif (r, c) == trap_state:
            print(" X ", end="")
        else:
            best_action = max(Q[(r, c)], key=Q[(r, c)].get)
            print(f" {best_action[0].upper()} ", end="")
    print()


Learned Policy:
 R  R  D  D 
 D  X  R  D 
 R  R  R  G 


In [47]:
import random

maze = [
    ['S', '.', '.', '.'],
    ['.', 'X', '.', '.'],
    ['.', '.', '.', 'G']
]

ROWS = len(maze)
COLS = len(maze[0])

start_state = (0, 0)
goal_state = (2, 3)
trap_state = (1, 1)

actions = ['up', 'down', 'left', 'right']
action_map = {
    'up': (-1, 0),
    'down': (1, 0),
    'left': (0, -1),
    'right': (0, 1)
}

# --------------------------
# Initialize Q-table
# --------------------------
Q = {}
for r in range(ROWS):
    for c in range(COLS):
        Q[(r, c)] = {a: 0 for a in actions}

alpha = 0.1
gamma = 0.8
epsilon = 0.2
episodes = 200


# --------------------------
# Reward function
# --------------------------
def get_reward(state):
    if state == goal_state:
        return 10
    if state == trap_state:
        return -10
    return -1


# --------------------------
# State transition
# --------------------------
def get_next_state(state, action):
    r, c = state
    dr, dc = action_map[action]
    nr, nc = r + dr, c + dc

    if nr < 0 or nr >= ROWS or nc < 0 or nc >= COLS:
        return state  # hit wall

    return (nr, nc)


# --------------------------
# Print learned policy
# --------------------------
def print_policy():
    for r in range(ROWS):
        for c in range(COLS):
            if (r, c) == goal_state:
                print(" G ", end="")
            elif (r, c) == trap_state:
                print(" X ", end="")
            else:
                best_action = max(Q[(r, c)], key=Q[(r, c)].get)
                print(f" {best_action[0].upper()} ", end="")
        print()
    print()


# --------------------------
# Print Q values
# --------------------------
def print_q_table():
    print("\nQ-table:")
    for r in range(ROWS):
        for c in range(COLS):
            print(f"State ({r},{c}): {Q[(r,c)]}")
    print()


# --------------------------
# Training loop
# --------------------------
for episode in range(episodes):
    state = start_state

    while state != goal_state and state != trap_state:

        # epsilon-greedy
        if random.uniform(0, 1) < epsilon:
            action = random.choice(actions)
        else:
            action = max(Q[state], key=Q[state].get)

        next_state = get_next_state(state, action)
        reward = get_reward(next_state)

        best_next_action = max(Q[next_state], key=Q[next_state].get)

        # Q update
        Q[state][action] += alpha * (
            reward + gamma * Q[next_state][best_next_action] - Q[state][action]
        )

        state = next_state

    # Print Q-table and policy after every episode
    print(f"=== After Episode {episode + 1} ===")
    print_policy()
    print_q_table()

print("=== Final Learned Policy ===")
print_policy()


=== After Episode 1 ===
 L  L  U  U 
 D  X  U  U 
 U  U  U  G 


Q-table:
State (0,0): {'up': -0.1, 'down': -0.1, 'left': 0, 'right': -0.1}
State (0,1): {'up': -0.1, 'down': -1.0, 'left': 0, 'right': 0}
State (0,2): {'up': 0, 'down': 0, 'left': 0, 'right': 0}
State (0,3): {'up': 0, 'down': 0, 'left': 0, 'right': 0}
State (1,0): {'up': -0.1, 'down': 0, 'left': 0, 'right': 0}
State (1,1): {'up': 0, 'down': 0, 'left': 0, 'right': 0}
State (1,2): {'up': 0, 'down': 0, 'left': 0, 'right': 0}
State (1,3): {'up': 0, 'down': 0, 'left': 0, 'right': 0}
State (2,0): {'up': 0, 'down': 0, 'left': 0, 'right': 0}
State (2,1): {'up': 0, 'down': 0, 'left': 0, 'right': 0}
State (2,2): {'up': 0, 'down': 0, 'left': 0, 'right': 0}
State (2,3): {'up': 0, 'down': 0, 'left': 0, 'right': 0}

=== After Episode 2 ===
 L  L  U  U 
 L  X  U  U 
 D  U  U  G 


Q-table:
State (0,0): {'up': -0.198, 'down': -0.19, 'left': -0.1, 'right': -0.1}
State (0,1): {'up': -0.1, 'down': -1.0, 'left': 0, 'right': 0}
State (0,2): {